In [1]:
print("hello")

hello


In [3]:
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.llms import ctransformers
from langchain_pinecone import PineconeVectorStore

c:\Users\krish\projects\med-bot\env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
from dotenv import load_dotenv
load_dotenv()
import os

In [10]:
pinecone_api=os.getenv("PINECONE_API_KEY")


In [8]:
def importdata(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    document=loader.load()
    return document

In [9]:
docs=importdata("data/")

In [10]:
def datachunking(data_extracted):
    chunker=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
    chunks=chunker.split_documents(data_extracted)
    return chunks

In [11]:
texts=datachunking(docs)

In [12]:
len(texts)

30789

In [1]:
def embeddingmodel():
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [4]:
embedder=embeddingmodel()

In [11]:

os.environ['PINECONE_API_KEY'] = pinecone_api
#PineconeVectorStore.from_texts([p.page_content for p in texts],embedding=embedder,index_name="med-bot")

In [12]:
from langchain_pinecone import PineconeVectorStore
import os

docsearch=PineconeVectorStore.from_existing_index(index_name="med-bot",embedding=embedder)

In [42]:
res=docsearch.similarity_search("what is acne?",k=4)

In [ ]:
print(res[0].page_content,res[1].page_content)
print(res[2].page_content,res[3].page_content)

In [15]:
prompttemplate="""
use the following pieces of context to answer the user's question
if you don't know the answer just say "not related to me" ,don't try to make up the answer
context={context}
question={question}
summerize the result with right words and return below
answer:""" 
pprompt=PromptTemplate(template=prompttemplate,input_variables=["context", "question"])

In [ ]:
load_dotenv()
os.getenv("openai_API_KEY")

In [36]:
from openai import AzureOpenAI
API_VERSION = "2024-02-01"
MODEL_NAME = "gpt-35-turbo"
ENDPOINT = os.getenv("ENDPOINT")
API_KEY = os.getenv("openai_API_KEY")
client = AzureOpenAI(
    azure_endpoint=ENDPOINT,
    api_key=API_KEY,
    api_version=API_VERSION,
)
from src.helper import modelResult
modelResult(client=client,docsearch=docsearch,question="who is prime mininster of india?")

'The given context does not provide information about the current Prime Minister of India. Therefore, I am unable to provide a detailed answer to your question.'

In [40]:
from src.helper import modelResult
modelResult(client=client,docsearch=docsearch,question="who is prime mininster of india?")


"As an AI language model, I can provide you with the information that as of September 2021, the Prime Minister of India is Narendra Modi. However, please note that political positions can change over time, so it's always a good idea to check for the most up-to-date information."

In [ ]:
print(prompttemplate.format(context="hello",question="acne"))

In [17]:
llm=ctransformers.CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={"max_new_tokens":512,
                          "temperature":0.5})

In [18]:

medbot=RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=docsearch.as_retriever(kwargs={"k":2}),
                              
                            chain_type_kwargs={"prompt": pprompt})

In [ ]:
response=medbot({"query":"what is acne?"})
print(response["result"])


In [20]:
medbot.invoke("acne")

{'query': 'acne', 'result': " Acneverymous drugs, I don's. \n"}

In [24]:
from langchain.chains import RetrievalQAWithSourcesChain
medbot2=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=docsearch.as_retriever())

In [ ]:
medbot2({"question":"what is aging"},return_only_outputs=True)